In [46]:
import pandas as pd

columns = ["ประเด็นที่ร้องเรียน", "หมวดหมู่ปัญหา (เลือก)", "พื้นที่ / ภูมิภาค", "หน่วยงาน\nที่เกี่ยวข้อง"]
data = pd.read_excel("complaints.xlsx", usecols=columns)
data.columns = ["text", "category", "region", "organization"]
data = data.dropna() # เอา missing value ออก
data = data.reset_index(drop=True)
data

,text,category,region,organization
0,ขอหน่วยงานที่เกี่ยวข้องวางท่อส่งน้ำสำหรับขยายเ...,น้ำ,สระบุรี,สปน.
1,ขอให้หน่วยงานที่เกี่ยวข้องดำเนินการแก้ไขปัญหาร...,อื่น ๆ,ทุกภูมิภาค,กต.
2,ขอให้ช่วยเหลือเยียวยาประชาชนที่ได้รับผลกระทบจา...,สาธารณภัย,สุพรรณบุรี,กษ.
3,ขอให้รัฐบาลทบทวนการแก้ไขปัญหาประชาคมกระเหรี่ยง...,ที่ดินทำกิน,เพชรบุรี,ทส.
4,ขอให้พิจารณาดำเนินการแก้ไขปัญหาการลักลอบเล่นพน...,อาชญากรรม ยาเสพติดและการพนัน,กรุงเทพมหานคร,ตร.
...,...,...,...,...
1952,ขอให้พิจารณากำกับดูแลและเร่งดำเนินการจัดทำร่าง...,การเมือง,ทุกภูมิภาค,สลค.
1953,ขอให้หน่วยงานที่เกี่ยวข้องบูรณาการการทำงานร่วม...,สวัสดิการ,ทุกภูมิภาค,กต.
1954,ขอให้หน่วยงานที่เกี่ยวข้องดำเนินการฟื้นฟูเศรษฐ...,เศรษฐกิจ การคลัง,ภาคตะวันออก,คค.
1955,ขอความอนุเคราะห์ให้จัดระบบประปาหมู่บ้าน ม.6 บ้...,น้ำ,เชียงใหม่,มท.


In [47]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.3, stratify=data["category"], random_state=0) # 🤔
train_data.shape[0], test_data.shape[0]

(1369, 588)

In [48]:
%pip install newmm_tokenizer

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from newmm_tokenizer.tokenizer import word_tokenize

def create_pipeline():
    return make_pipeline(
        # CountVectorizer(), # 🤔
        # TfidfVectorizer(), # 🤔
        # TfidfVectorizer(ngram_range=(1, 3), min_df=5, max_df=0.5), # 🤔
        CountVectorizer(tokenizer=word_tokenize), # 🤔 แยกคำออกเป็น column
        # TfidfVectorizer(tokenizer=word_tokenize, ngram_range=(1, 3), min_df=5, max_df=0.5), # 🤔

        # SelectKBest(k=1000), # 🤔 เอา 1000 features ที่ดีที่สุดมาใช้
        
        MultinomialNB()
        # AdaBoostClassifier(estimator=LogisticRegression(), n_estimators=10, random_state=0) # 🤔
        # RandomForestClassifier(n_estimators=10, random_state=0) # 🤔
    )

pipeline = create_pipeline()
pipeline.fit(train_data["text"], train_data["category"])
pipeline.score(train_data["text"], train_data["category"])

d:\Mook\262_ML\lab\.venv\lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.7355734112490869

In [50]:
pipeline.steps[-1][1].n_features_in_

5730

In [51]:
pipeline.score(test_data["text"], test_data["category"])

0.5153061224489796

In [72]:
pipeline.predict(["น้ำท่วม"])[0]   # ทายว่าเป็น column ไหน

np.str_('สาธารณภัย')

In [53]:
pipeline.predict(["ถนนทรุด"])[0]

np.str_('โครงสร้างพื้นฐานและสาธารณูปโภค')

In [57]:
# แนะนำคำตอบ
predicted = pipeline.predict_proba(["ถนนทรุด"])[0]
predicted

array([1.37341547e-03, 1.62997949e-04, 8.20843096e-02, 7.30108134e-04,
       1.99776217e-03, 2.72194596e-02, 2.47787543e-03, 2.38529696e-03,
       2.39528903e-03, 6.33314697e-03, 3.24298542e-03, 8.73826430e-04,
       1.13739562e-02, 2.06691377e-02, 2.04958899e-03, 6.03198803e-03,
       1.10578342e-03, 4.49198052e-03, 4.64284290e-04, 2.25705411e-02,
       3.96800700e-03, 1.98016765e-01, 4.40140762e-03, 3.02502941e-03,
       6.51061203e-03, 9.80854037e-03, 3.82935892e-02, 1.32516556e-03,
       1.24960457e-03, 8.61928911e-03, 2.00292804e-03, 5.19330535e-01,
       1.14580527e-03, 2.26898828e-03])

In [55]:
from collections import OrderedDict

OrderedDict(sorted(zip(pipeline.classes_, predicted), key=lambda x: x[1])[::-1])

OrderedDict([(np.str_('โครงสร้างพื้นฐานและสาธารณูปโภค'),
              np.float64(0.5193305354915263)),
             (np.str_('สาธารณภัย'), np.float64(0.1980167646618376)),
             (np.str_('การคมนาคม การจราจรหรืออุบัติเหตุ'),
              np.float64(0.08208430961517234)),
             (np.str_('อื่น ๆ'), np.float64(0.03829358920736119)),
             (np.str_('การบริหารราชการแผ่นดิน'),
              np.float64(0.027219459595867055)),
             (np.str_('ศิลปะ วัฒนธรรม ศาสนา'),
              np.float64(0.022570541063418305)),
             (np.str_('น้ำ'), np.float64(0.02066913766988561)),
             (np.str_('ที่ดินทำกิน'), np.float64(0.011373956239822219)),
             (np.str_('อาชญากรรม ยาเสพติดและการพนัน'),
              np.float64(0.0098085403656902)),
             (np.str_('เศรษฐกิจ การคลัง'), np.float64(0.008619289114140187)),
             (np.str_('หนี้สิน เศรษฐกิจฐานราก'),
              np.float64(0.006510612027674902)),
             (np.str_('การเมือง'), np.float6